In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import math
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import torchvision
from torchvision.utils import save_image
from IPython.display import Image

In [2]:
xls = pd.ExcelFile('Master Data Sheet Organization.xlsx')
data_df = pd.read_excel(xls, 'Master Sheet - M1')
new_header = data_df.iloc[0] #grab the first row for the header
data_df = data_df[1:] #take the data less the header row
data_df.columns = new_header #set the header row as the df header
data_df = data_df.loc[(data_df['Picture?'] == 'Y')].reset_index(drop=True)
data_df.reset_index(drop=True, inplace=True)
data_df

,NaN,Country,District,Sentinel Site,Date,MOSQUITO ID (Team),Morph ID Genus,Morph ID Species,PCR Number (Country Specific),PCR ID Genus,PCR ID Species,Sibling Species,Picture?,Prototype,Abdomen Status,Sex,Parity,Eliza Results
0,6.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0006,Anopheles,funestus,498,NaN,funestus,NaN,Y,1409,SG,F,NaN,NaN
1,8.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0008,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
2,9.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0009,Anopheles,funestus,NaN,NaN,funestus,NaN,Y,NaN,G,F,NaN,NEG
3,10.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0010,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,G,F,NaN,NEG
4,11.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0011,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,NaN,NaN,NaN,NEG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1646.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0295,Anopheles,gambiae,MI2100295,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1405,1647.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0296,Anopheles,rufipes,MI2100296,Anopheles,NaN,arabiensis,Y,NaN,UF,F,NaN,NaN
1406,1648.0,Zambia,Matcha,Chidakwa,2021-05-10 00:00:00,Z-MA-01-0297,Anopheles,rufipes,MI2100297,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1407,1649.0,Zambia,Matcha,Chidakwa,2021-05-09 00:00:00,Z-MA-01-0298,Anopheles,longipalps,MI2100298,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN


In [3]:
df_a = data_df[(data_df['Morph ID Genus'] == 'Anopheles') | (data_df['Morph ID Genus'] == 'anopheles') | (data_df['Morph ID Genus'] == 'Anopheles ')]
print(df_a.shape)
df_b = data_df[(data_df['Morph ID Genus'] == 'Culex')]
print(df_b.shape)
df_c = data_df[(data_df['Morph ID Genus'] != 'Culex') & (data_df['Morph ID Genus'] != 'Anopheles') & (data_df['Morph ID Genus'] != 'anopheles')& (data_df['Morph ID Genus'] != 'Anopheles ')]
print(df_c.shape)

(1000, 18)
(224, 18)
(185, 18)


In [4]:
# use PCR results if avaliable
df = pd.DataFrame()
ID = []
genus = []
species = []
for i in range(data_df.shape[0]):
    ID.append(data_df.loc[i,'MOSQUITO ID (Team)'])
    genus_name = data_df.loc[i,'Morph ID Genus']
    species_name = data_df.loc[i,'Morph ID Species']
    if genus_name == 'Anopheles' or genus_name == 'anopheles' or genus_name == 'Anopheles ':
        if pd.notna(data_df.loc[i,'PCR ID Genus']) == True:
            genus_name = data_df.loc[i,'PCR ID Genus']
        if pd.notna(data_df.loc[i,'PCR ID Species']) == True:
            species_name = data_df.loc[i,'PCR ID Species']
    genus.append(genus_name)
    species.append(species_name)
df['ID'] = ID
df['genus'] = genus
df['species'] = species
df = df.dropna(subset=['genus'],how='any').reset_index(drop=True)
df

,ID,genus,species
0,U-SR-01-0006,Anopheles,funestus
1,U-SR-01-0008,Anopheles,funestus
2,U-SR-01-0009,Anopheles,funestus
3,U-SR-01-0010,Anopheles,funestus
4,U-SR-01-0011,Anopheles,funestus
...,...,...,...
1350,Z-MA-01-0295,Anopheles,gambiae
1351,Z-MA-01-0296,Anopheles,rufipes
1352,Z-MA-01-0297,Anopheles,rufipes
1353,Z-MA-01-0298,Anopheles,longipalps


In [5]:
df_with_label = pd.DataFrame()
label = []
for i in range(df.shape[0]):
    genus = df.loc[i,'genus']
    species = df.loc[i,'species']
    if genus == 'Anopheles' or genus == 'anopheles' or genus == 'Anopheles ':
        if species == 'funestus':
            label.append(0)
        elif species == 'gambiae' or species == 'Gambiae' or species == 'gambie':
            label.append(1)
        else:
            label.append(2)
    elif genus == "Culex" or genus == "Culex ":
        label.append(3)
    else:
        label.append(4)
        
df_with_label['ID'] = df['ID']
df_with_label['label'] = label
df_with_label

,ID,label
0,U-SR-01-0006,0
1,U-SR-01-0008,0
2,U-SR-01-0009,0
3,U-SR-01-0010,0
4,U-SR-01-0011,0
...,...,...
1350,Z-MA-01-0295,1
1351,Z-MA-01-0296,2
1352,Z-MA-01-0297,2
1353,Z-MA-01-0298,2


In [6]:
df1 = df_with_label[df_with_label['label'] == 0]
df2 = df_with_label[df_with_label['label'] == 1]
df3 = df_with_label[df_with_label['label'] == 2]
df4 = df_with_label[df_with_label['label'] == 3]
df5 = df_with_label[df_with_label['label'] == 4]
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(356, 2)
(293, 2)
(351, 2)
(332, 2)
(23, 2)


In [7]:
train_data = pd.concat(
    [df1,df2,df3,df4, df5], ignore_index=True
)

print(train_data.shape)

(1355, 2)


In [8]:
train_data

,ID,label
0,U-SR-01-0006,0
1,U-SR-01-0008,0
2,U-SR-01-0009,0
3,U-SR-01-0010,0
4,U-SR-01-0011,0
...,...,...
1350,U-SR-01-0712,4
1351,U-SR-01-0716,4
1352,G-TM-05-0001,4
1353,G-TM-05-0002,4


In [9]:
from torchvision import transforms
mosquito_transforms = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize([300,300]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [10]:
from PIL import Image, ImageOps
def make_data(d):
    data_path = "/home/shrutihegde/Documents/Shruti/Species/Data/cropped/crop_m1_pad/"
    data = []
    label = []
    paths = []
    F_count = 0
    G_count = 0
    Aother_count = 0
    C_count = 0
    Other_count = 0
    df = pd.DataFrame()
    for i in range(d.shape[0]):
        for f in os.listdir(data_path):
            if re.match(d.loc[i,'ID'], f) and f.endswith('a.jpg'):
                path = os.path.join(data_path, f)
                paths.append(path)
                image = Image.open(path)
                #image = ImageOps.exif_transpose(image)
                #image = mosquito_transforms(image)
                data.append(image)
                if d.loc[i,'label'] == 0:
                    label.append(0)
                    F_count += 1
                elif d.loc[i,'label'] == 1:
                    label.append(1)
                    G_count += 1
                elif d.loc[i,'label'] == 2:
                    label.append(2)
                    Aother_count += 1    
                elif d.loc[i,'label'] == 3:
                    label.append(3)
                    C_count += 1
                elif d.loc[i,'label'] == 4:
                    label.append(4)
                    Other_count += 1
    stacked_data = torch.stack(data)
    df['path'] = paths
    df['label'] = label  
                
    return stacked_data, label, df, F_count, G_count, Aother_count, C_count, Other_count 

In [13]:
d, label, path_df, _,_,_,_,_= make_data(train_data)

In [14]:
import cv2
def split_into_folders(path_df):
    for ind in path_df.index:
        if(path_df['label'][ind]==0):
            path=path_df['path'][ind]
            image_cv2= cv2.imread(path_df['path'][ind])
            filename=path.split('/')[-1].split('_')[0]
            result=cv2.imwrite(r'/home/shrutihegde/Documents/Shruti/Species/Data/cropped/Images_crop/AF/'+filename+'.jpg', image_cv2)
            if result==False:
                print("Error saving file")

        elif(path_df['label'][ind]==1):
            path=path_df['path'][ind]
            image_cv2= cv2.imread(path_df['path'][ind])
            filename=path.split('/')[-1].split('_')[0]
            result=cv2.imwrite(r'/home/shrutihegde/Documents/Shruti/Species/Data/cropped/Images_crop/AG/'+filename+'.jpg', image_cv2)
            if result==False:
                print("Error saving file")
        elif(path_df['label'][ind]==2):
            path=path_df['path'][ind]
            image_cv2= cv2.imread(path_df['path'][ind])
            filename=path.split('/')[-1].split('_')[0]
            result=cv2.imwrite(r'/home/shrutihegde/Documents/Shruti/Species/Data/cropped/Images_crop/A_other/'+filename+'.jpg', image_cv2)
            if result==False:
                print("Error saving file")
        elif(path_df['label'][ind]==3):
            path=path_df['path'][ind]
            image_cv2= cv2.imread(path_df['path'][ind])
            filename=path.split('/')[-1].split('_')[0]
            result=cv2.imwrite(r'/home/shrutihegde/Documents/Shruti/Species/Data/cropped/Images_crop/Culex/'+filename+'.jpg', image_cv2)
            if result==False:
                print("Error saving file")
        else:
            path=path_df['path'][ind]
            image_cv2= cv2.imread(path_df['path'][ind])
            filename=path.split('/')[-1].split('_')[0]
            result=cv2.imwrite(r'/home/shrutihegde/Documents/Shruti/Species/Data/cropped/Images_crop/other/'+filename+'.jpg', image_cv2)
            if result==False:
                print("Error saving file")  

In [15]:
split_into_folders(path_df)

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

...

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe

File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved successfully
File saved succe